[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OlivierGeorgeon/Developmental-AI-Lab/blob/master/docs/agent2.ipynb)

# AGENT 2 - THE AGENT WHO THRIVED ON GOOD VIBES

# Learning objectives

Upon completing this lab, you will be able to implement agents driven by a type of intrinsic motivation called 'interactional motivation.' This refers to the drive to engage in sensorimotor interactions that have a positive valence while avoiding those that have a negative valence.

# Setup
## Define the Agent class

In [1]:
class Agent:
    def __init__(self, _valences):
        """ Creating our agent """
        self._valences = _valences
        self._action = None
        self._predicted_outcome = None

    def action(self, _outcome):
        """ tracing the previous cycle """
        if self._action is not None:
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {_outcome}, " 
                  f"Prediction: {self._predicted_outcome == _outcome}, Valence: {self._valences[self._action][_outcome]}")

        """ Computing the next action to enact """
        # TODO: Implement the agent's decision mechanism
        self._action = 0
        # TODO: Implement the agent's anticipation mechanism
        self._predicted_outcome = 0
        return self._action


## Environment1 class

In [2]:
class Environment1:
    """ In Environment 1, action 0 yields outcome 0, action 1 yields outcome 1 """
    def outcome(self, _action):
        # return int(input("entre 0 1 ou 2"))
        if _action == 0:
            return 0
        else:
            return 1

## Environment2 class

In [3]:
class Environment2:
    """ In Environment 2, action 0 yields outcome 1, action 1 yields outcome 0 """
    def outcome(self, _action):
        if _action == 0:
            return 1
        else:
            return 0

## Define the valence of interactions

In [4]:
valences = [[-1, 1], 
            [1, -1]]

The valence table specifies the valence of each interaction. An interaction is a tuple (action, outcome):

|| outcome 0 | outcome 1 |
|---|---|---|
| action 0 | -1 | 1 |
| action 1 | 1 | -1 |

## Instantiate the agent

In [5]:
a = Agent(valences)

## Instantiate the environment 

In [6]:
e = Environment1()

## Test run the simulation

In [7]:
outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1


Observe that, on each interaction cycle, the agent is mildly satisfied. On one hand, the agent made correct predictions, on the other hand, it experienced negative valence.

# PRELIMINARY EXERCISE

Execute the agent in Environment2. Observed that it obtains a positive valence. 

Modify the valence table to give a positive valence when the agent selects action `0` and obtains outcome `0`.
Observe that this agent obtains a positive valence in Environment1. 

# ASSIGNMENT

Implement Agent2 that selects actions that, it predicts, will result in an interaction that have a positive valence. 

Only when the agent gets bored does it select an action which it predicts to result in an interaction that have a negative valence. 

In the trace, you should see that the agent learns to obtain a positive valence during several interaction cycles.
When the agent gest bored, it occasionnaly selects an action that may result in a negative valence. 

## Create Agent2 by overriding the class Agent

In [8]:
class Agent2(Agent):
    def __init__(self, _valences):
        """Creating our hedonist agent"""
        super().__init__(_valences)
        # Memory: stores the last observed outcome for each action
        self.memory = {}
        # Counter for consecutive correct predictions
        self.correct_count = 0
        # Boredom threshold
        self.boredom_threshold = 4
        
    def action(self, _outcome):
        """Tracing the previous cycle"""
        if self._action is not None:
            # Update memory with the observed outcome
            self.memory[self._action] = _outcome
            
            # Check if prediction was correct
            satisfied = (self._predicted_outcome == _outcome)
            
            # Update correct prediction counter
            if satisfied:
                self.correct_count += 1
            else:
                self.correct_count = 0
            
            # Calculate valence
            valence = self._valences[self._action][_outcome]
            
            # Check for boredom
            bored = (self.correct_count >= self.boredom_threshold)
            
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, "
                  f"Outcome: {_outcome}, Prediction: {satisfied}, Valence: {valence}, Bored: {bored}")
        
        """Computing the next action to enact"""

        #  Implement the agent's decision mechanism
        if self.correct_count >= self.boredom_threshold:
            # Bored: try a different action
            self._action = 1 - self._action
            self.correct_count = 0
        else:
            # Not bored: choose the action with the best anticipated valence
            best_action = None
            best_valence = -float('inf')
            
            for action in [0, 1]:
                if action in self.memory:
                    predicted_outcome = self.memory[action]
                    predicted_valence = self._valences[action][predicted_outcome]
                    
                    if predicted_valence > best_valence:
                        best_valence = predicted_valence
                        best_action = action
            
            if best_action is not None:
                self._action = best_action
            elif self._action is None:
                self._action = 0

        # Implement the agent's anticipation mechanism
        if self._action in self.memory:
            self._predicted_outcome = self.memory[self._action]
        else:
            self._predicted_outcome = 0
        
        return self._action

## Test your Agent2 in Environment1

In [14]:
valences = [[-1, 1],
            [1, -1]]

a = Agent2(valences)
e = Environment1()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: True
Action: 1, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: True
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Predic

## Test your Agent2 in Environment2

In [10]:
a = Agent2(valences)
e = Environment2()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: True
Action: 1, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: True
Action: 1, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 1, Out

# Test your agent with a different valence table

Note that, depending on the valence that you define, it may be impossible for the agent to obtain a positive valence in some environments. 

In [16]:
valences = [[-1, 1],
            [-1, 1]]
#  agent 2 environment 1
a = Agent2(valences)
e = Environment1()
outcome = 0
for i in range(20):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: True
Action: 1, Prediction: 0, Outcome: 1, Prediction: False, Valence: 1, Bored: False
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: True
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 1, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 1, Prediction: 1

## Report 

Explain what you programmed and what results you observed. Export this document as PDF including your code, the traces you obtained, and your explanations below (no more than a few paragraphs):

## Rapport — Agent 2

Pour cette partie, nous avons étendu le comportement de l’agent en intégrant la valence comme critère de décision.
L’agent ne se contente plus d’anticiper les outcomes. Il choisit désormais l’action qui maximise la valence prédite à partir de la table `self._valences`.
Lorsqu’il s’ennuie après 4 prédictions correctes consécutives, il sélectionne une autre action, même si celle-ci est associée à une valence négative.

### Environment 1

Dans Environment 1, les valences définies (`valences = [[-1, 1], [1, -1]]`) attribuent une valence négative à toutes les interactions possibles. Quelle que soit l’action effectuée (`0` ou `1`), l’outcome obtenu conduit systématiquement à une valence de `-1`. Ainsi, il est impossible pour l’agent d’obtenir une valence positive dans cet environnement avec cette configuration.

L’agent prédit correctement les outcomes dès le début, mais la valence reste négative. Lors du choix d’action, lorsqu’il compare les valences anticipées, il rencontre l’action `0` en premier et conserve cette action puisqu’elle offre la même valence que l'action `1`. Le code ne prévoit pas de changer d’action en cas d’égalité, ce qui explique pourquoi il reste sur l'action `0` tant qu’il n’est pas ennuyé.

Il alterne ensuite ses actions uniquement en réponse à l’ennui détecté au 4ᵉ cycle (`Bored = True`), avant de revenir sur l'action `0` lors du prochain choix. Ce comportement engendre un schéma cyclique stable : 4 cycles corrects avec une valence constante de `-1`, ennui au 4ᵉ, changement d’action au 5ᵉ, puis retour sur l'action `0` et répétition du même pattern.


### Environment 2

Dans Environment 2, avec la table de valences `valences = [[-1, 1], [1, -1]]`, l’agent commence par exécuter l’action `0` avec une anticipation initiale incorrecte (`Prediction: 0`), ce qui entraîne une erreur dès le 1ᵉʳ cycle. Dès le 2ᵉ cycle, il ajuste sa prédiction à `1`, ce qui correspond à l’outcome réel (`0 → 1`) et lui permet d’obtenir une valence positive de `+1`.

L’action `1` offre également une valence de `+1` (`1 → 0`), mais l’agent privilégie l’action `0` car le mécanisme de décision explore les actions dans l’ordre et sélectionne la première qui présente la meilleure valence. Comme les deux actions ont la même valence positive, l'action `0` est systématiquement retenue. L’agent ne bascule donc sur l’action `1` que lorsqu’il détecte l’ennui au 4ᵉ cycle (`Bored = True`).

Après avoir changé d’action à cause de l’ennui, l’agent obtient une valence positive avec `1`, identique à celle de l'action `0`.
Cependant, à cause de notre logique de code, au cycle suivant, il revient systématiquement sur l'action`0`.
Le schéma devient ainsi régulier avec 4 cycles sur l'action `0`, ennui, passage à l'action à `1`, puis retour à l'action `0`.



### Environment 1 avec valence différente `valences = [[-1, 1], [-1, 1]]`

Au départ l’agent ne sait pas que l’action `1` peut donner une valence positive avec l’outcome `1`.
N’ayant encore rien mémorisé pour l’action `1`, il reste sur l’action `0`, ce qui lui donne une valence négative de `-1` pendant 4 cycles.
Lorsque l’ennui apparaît au 4ᵉ cycle, il essaie l’action `1`, obtient une valence positive de `+1` et mémorise cette association favorable.

Dès que cette information est en mémoire, le mécanisme de décision privilégie `1` (car elle maximise la valence anticipée) et l’agent y reste jusqu’au prochain ennui.
Lorsqu’il s’ennuie à nouveau, il fait un bref passage par l'action `0` (valence `-1`) puis revient rapidement sur l'action `1` qui reste la meilleure option.

Schéma observé
4 cycles sur `0` (valence `-1`), puis ennui, puis passage à `1` (valence `+1`).
Après ce changement, l’agent maintient ses cycles sur `1` jusqu’au prochain ennui, fait un bref détour par `0`, puis revient rapidement sur `1` pour reprendre un cycle stable avec la valence positive.

### Conclusion 1

Nous avons réussi à mettre en place tout ce qui était demandé dans le TP.
L’agent apprend à prédire les outcomes, cherche les interactions à valence positive et change d’action lorsqu’il s’ennuie.

On peut aussi rendre l’agent plus flexible.
Par exemple, il pourrait choisir au hasard entre les actions qui ont la même valence max pour éviter de faire toujours le même choix.

Ensuite, on peut ajuster son comportement face à l’ennui.
Il pourrait s’ennuyer plus vite au début pour explorer rapidement et découvrir les meilleures valences,
puis réduire son ennui une fois une bonne interaction trouvée, surtout dans des environnements avec plusieurs actions possibles.




